<a href="https://colab.research.google.com/github/Sujangyawali/Fraud_Detection/blob/master/after_exam_seven_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras==2.1.0 
!pip install tensorflow==1.15.0

     |████████████████████████████████| 307kB 16.4MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.0 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 412.3MB 37kB/s 
     |████████████████████████████████| 3.8MB 51.7MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 512kB 52.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ebb77118fb308b0594634043fa9bbf8e1af9547f3610b477d9f81c78c3580656
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully unin

In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from sklearn.model_selection import KFold
from imgaug import augmenters as iaa

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
!python setup.py install


Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 38.66 MiB/s, done.
Resolving deltas: 100% (561/561), done.
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/li

In [ ]:
#part of localization
# example of inference with a pre-trained coco model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn.utils import Dataset
from mrcnn.utils import extract_bboxes
from mrcnn.visualize import display_instances
from mrcnn.model import MaskRCNN

Using TensorFlow backend.


In [ ]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset

class KangarooDataset(Dataset):
  def load_dataset(self,dataset_dir,is_train=True):
    self.add_class("dataset",1,"firetruck")
    self.add_class("dataset",2,"ambulance")
    self.add_class("dataset",3,"hiace")
    self.add_class("dataset",4,"bus")
    self.add_class("dataset",5,"truck")
    self.add_class("dataset",6,"car")
    self.add_class("dataset",7,"jeep")

    images_dir=dataset_dir + '/images/'
    annotations_dir=dataset_dir +'/annotation/'

    a=0
    for filename in listdir(images_dir):
      a=a+1
      image_id=filename[:-4]

      if is_train and a <28:
        continue
      if not is_train and a>28:
        continue
      img_path=images_dir + filename
      ann_path=annotations_dir+image_id+'.xml'

      self.add_image('dataset',image_id=image_id,path=img_path,annotation=ann_path)

  def extract_boxes(self, filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each object
    boxes = list()
    for object in root.findall('.//object'):
        box_class_list = list()
        #find bbox coordinates
        for box in object.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            box_class_list.append(coors)
        #get the name of the object class corresponding to the bbox
        for name in object.findall('.//name'):
            box_class_list.append(name.text)
        #append the box coors and respective name to a list
        boxes.append(box_class_list)
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height



  def load_mask(self, image_id):
    # get details of image
    info = self.image_info[image_id]
    # define box file location, here the annotation dir in project dir
    path = info['annotation']
    # load XML
    boxes, w, h = self.extract_boxes(path)
    # create one array for all masks, each on a different channel
    masks = zeros([h, w, len(boxes)], dtype='uint8')
    # create masks
    class_ids = list()
    for i in range(len(boxes)):
        box = boxes[i][0]
        row_s, row_e = box[1], box[3]
        col_s, col_e = box[0], box[2]
        masks[row_s:row_e, col_s:col_e, i] = 1
        class_ids.append(self.class_names.index(boxes[i][1]))

    return masks, asarray(class_ids, dtype='int32') 


    def image_reference(self,image_id):
      info=self.image_info[image_id]
      return info['path']


# train set
train_set = KangarooDataset()
train_set.load_dataset('/content/drive/MyDrive/for_seven_class', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))


# test/val set
test_set = KangarooDataset()
test_set.load_dataset('/content/drive/MyDrive/for_seven_class', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
        

Train: 199
Test: 28


In [ ]:
from mrcnn.config import Config
# define a configuration for the model
class FiretruckConfig(Config):
	# Give the configuration a recognizable name
	NAME = "sevenclass_cfg"
	# Number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 7
	GPU_COUNT=1
	# Number of training steps per epoch
	STEPS_PER_EPOCH=110

# prepare config
config = FiretruckConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                20
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
import mrcnn.model as modellib
# define the model
model = modellib.MaskRCNN(mode='training', model_dir='./', config=config)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
    #downloading weight into the current directory

In [ ]:
# load weights (mscoco)
model.load_weights('/content/Mask_RCNN/mask_rcnn_coco.h5', by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# train weights (output layers or 'heads')
history=model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=20, layers='all')
new_history = model.keras_model.history.history


Starting at epoch 0. LR=0.001

Checkpoint Path: ./sevenclass_cfg20210206T1908/mask_rcnn_sevenclass_cfg_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b      

ERROR:root:Error processing image {'id': 'vid26.mp4frame195', 'source': 'dataset', 'path': '/content/drive/MyDrive/for_seven_class/images/vid26.mp4frame195.jpg', 'annotation': '/content/drive/MyDrive/for_seven_class/annotation/vid26.mp4frame195.xml'}
Traceback (most recent call last):
  File "/content/Mask_RCNN/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/Mask_RCNN/mrcnn/model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-9-d8107bf9b7c4>", line 70, in load_mask
    boxes, w, h = self.extract_boxes(path)
  File "<ipython-input-9-d8107bf9b7c4>", line 37, in extract_boxes
    tree = ElementTree.parse(filename)
  File "/usr/lib/python3.6/xml/etree/ElementTree.py", line 1196, in parse
    tree.parse(source, parser)
  File "/usr/lib/python3.6/xml/etree/ElementTree.py", line 586, in parse
    source = open(source, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '

 12/110 [==>...........................] - ETA: 3:56:51 - loss: 2.4451 - rpn_class_loss: 0.0122 - rpn_bbox_loss: 0.3981 - mrcnn_class_loss: 0.5979 - mrcnn_bbox_loss: 0.8146 - mrcnn_mask_loss: 0.6223

In [ ]:
epochs = range(1, len(new_history['loss'])+1)
pd.DataFrame(new_history, index=epochs)

NameError: ignored